In [2]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

자료를 벡터 db에 넣기

### 수강신청 웹크롤링

In [ ]:
#https://web.kangnam.ac.kr/menu/f19069e6134f8f8aa7f689a4a675e66f.do?paginationInfo.currentPageNo=1&searchMenuSeq=0&searchType=ttl&searchValue=%EC%88%98%EA%B0%95

#### 웹크롤링 pdf 다운로더

In [26]:
import requests
from bs4 import BeautifulSoup
import os

# 웹페이지 URL 입력
url = 'https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do'

# HTTP 요청을 보내 웹페이지 HTML 가져오기
response = requests.get(url)
response.raise_for_status()  # 요청이 성공했는지 확인

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(response.content, 'html.parser')

# 모든 PDF 링크 찾기
pdf_links = soup.find_all('a', href=lambda href: href and href.endswith('.pdf'))

# PDF 파일을 저장할 디렉토리 생성
os.makedirs('pdf_files', exist_ok=True)

# 각 PDF 파일을 다운로드하여 저장
for link in pdf_links:
    pdf_url = link['href']
    if not pdf_url.startswith('http'):  # 상대 경로인 경우
        pdf_url = url + pdf_url
    pdf_response = requests.get(pdf_url)
    pdf_response.raise_for_status()
    
    # 파일 이름 추출
    pdf_filename = os.path.join('pdf_files', pdf_url.split('/')[-1])
    
    # PDF 파일 저장
    with open(pdf_filename, 'wb') as pdf_file:
        pdf_file.write(pdf_response.content)
    
    print(f'Downloaded: {pdf_filename}')

print('All PDFs have been downloaded.')


All PDFs have been downloaded.


#### json(다중 링크) 형태

In [8]:
import json
import os

# JSON 파일이 저장된 디렉토리 경로
json_directory = './data/web_crawling_links'
json_file = 'cources.json'

# JSON 파일에서 URL과 제목 리스트 추출
def get_url_title_pairs_from_json(directory, filename):
    file_path = os.path.join(directory, filename)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        url_title_pairs = [(url, title) for category in data.values() for title, url in category.items()]
    return url_title_pairs

# URL과 제목 리스트 생성
url_title_pairs = get_url_title_pairs_from_json(json_directory, json_file)


In [9]:
import requests
from bs4 import BeautifulSoup

# 크롤링한 텍스트를 저장할 디렉토리 경로
text_directory = './crawled_texts'
os.makedirs(text_directory, exist_ok=True)

# 파일명으로 사용할 유효한 문자열 생성
def create_valid_filename(title):
    # 파일명으로 사용 불가능한 문자 제거
    return "".join([c if c.isalnum() else "_" for c in title])

# URL을 통해 웹 페이지를 크롤링하고 텍스트 저장
def crawl_and_save_text(url, title, directory):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 웹 페이지의 텍스트 추출 (여기서는 <body> 태그 내의 텍스트를 추출)
        text = soup.get_text(separator='\n', strip=True)
        
        # 파일명 생성 (제목을 이용하여 고유한 파일명 생성)
        valid_filename = create_valid_filename(title)
        file_name = os.path.join(directory, f"{valid_filename}.txt")
        
        # 텍스트 파일로 저장
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(text)
            
        print(f"Successfully saved text from {url} to {file_name}")
    except Exception as e:
        print(f"Failed to crawl {url}: {e}")

# 모든 URL과 제목에 대해 크롤링 및 텍스트 저장
for url, title in url_title_pairs:
    crawl_and_save_text(url, title, text_directory)


Successfully saved text from https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do?encMenuSeq=a5a508e7e77c5c739b990f33188efdac to ./crawled_texts/강의계획서.txt
Successfully saved text from https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do?encMenuSeq=0eb6196506a4fcd8504a16f2fdcfb011 to ./crawled_texts/수강신청.txt
Successfully saved text from https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do?encMenuSeq=2c831137273abadb3d773098e872f607 to ./crawled_texts/계절수업.txt
Successfully saved text from https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do?encMenuSeq=d2fca573c753f30f9ae5c79dd740bdcd to ./crawled_texts/재수강.txt
Successfully saved text from https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do?encMenuSeq=0c1a4788f43e44817e2cb11ffca2a77d to ./crawled_texts/결석조치.txt
Successfully saved text from https://web.kangnam.ac.kr/menu/fd8c126ac0e81458620beb18302bc271.do?encMenuSeq=84ac874658806a5357608340a8314f99 to ./crawled_texts/시

### Embedding

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
from glob import glob

# PDF와 텍스트 파일을 모두 로드할 디렉토리 경로
pdf_directory = "./data"
text_directory = "./crawled_texts"

# PDF 파일들을 로드하여 분할한 뒤 텍스트를 벡터로 변환하여 DB에 저장
def load_and_index_documents(pdf_directory, text_directory):
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)

    # 텍스트 파일들을 로드하여 분할
    text_files = glob(os.path.join(text_directory, '*.txt'))
    for text_file in text_files:
        with open(text_file, 'r', encoding='utf-8') as file:
            text = file.read()
            # 텍스트를 Document 객체로 변환
            text_document = Document(page_content=text, metadata={"source": text_file})
            documents.append(text_document)

    # 분할된 텍스트를 벡터로 변환하여 ChromaDB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
    retriever = vectordb.as_retriever()
    
    return retriever

# Load and index documents from both PDF and text files
retriever = load_and_index_documents(pdf_directory, text_directory)


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)


### 프롬프트

In [11]:
SYS_PROMPT = f"""
    너는 사용자가 강남대학교에서 졸업할 수 있는지 등을 물어보았을때 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    사용자의 수강정보는 강남대학교의 졸업요건과 함께 데이터베이스에 같이 저장되어 있어.
    너는 
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
"""

모델 선언

In [12]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
1.강남대학교 AI로드맵을 위한 참고 자료.:{context}
2.사용자 입력 메세지 : {question}
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
나 졸업 언제쯤 가능할꺼같아? 한글로 대답해줘
"""   # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke("input_message"))


> 안녕하세요! 강남대학교 학사지원 인공지능 챗봇입니다. 졸업 요건이나 수강 정보에 대해 궁금한 점이 있으시면 언제든지 물어보세요. 
> 
> 사용자님의 수강 정보와 강남대학교의 졸업 요건을 바탕으로 성실하게 답변해드리겠습니다. 
> 
> 무엇을 도와드릴까요? 예를 들어, "졸업 요건을 충족했는지 확인해 주세요" 또는 "현재 수강 중인 과목이 졸업 요건에 포함되는지 알고 싶어요"와 같은 질문을 하실 수 있습니다.
> 
> 사용자 입력 메시지를 입력해 주세요.

In [6]:
import gradio as gr
from PyPDF2 import PdfReader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import json

template = '''Answer the question based only on the following context: {context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# 모델 설정
model = ChatOpenAI(model='gpt-4', temperature=0)

# 문서들을 형식화하는 함수 정의
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# PDF 파일에서 텍스트를 추출하는 함수
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# 사용자의 질문에 대한 답변을 생성하는 함수
def answer_question(pdf_path, question):
    # PDF에서 텍스트 추출
    context = extract_text_from_pdf(pdf_path)
    
    # 프롬프트 생성
    formatted_prompt = prompt.format_prompt(context=context, question=question)
    
    # System 및 Human 메시지 생성
    system_message = SystemMessage(content=SYS_PROMPT)
    user_message = HumanMessage(content=formatted_prompt.to_string())
    
    # 모델 호출
    response = model([system_message, user_message])
    return response.content

# 그라디오 인터페이스 설정
iface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.File(label="Upload PDF", type="filepath"),
        gr.Textbox(label="Question", lines=2)
    ],
    outputs="text",
    title="PDF QA Chatbot",
    description="Upload a PDF and ask questions about its content."
)

# 그라디오 인터페이스 실행
iface.launch()


/opt/anaconda3/envs/svp_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/envs/svp_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
